## Cargar y parsear dataset

In [1]:
import pandas as pd

In [2]:
train = '/content/AF_train.xlsx'
test = '/content/AF_test.xlsx'

In [3]:
df_train = pd.read_excel(train)

In [4]:
df_train.head()

,sequence,label
0,ACFKFLGECGAVPFPGTNADCTSCCVGNFGSAVCAGRVEVEGGVKH...,1
1,ACFTFLGECGPEPFTGSNADCLACCVALYKSPPVCAGRVEGVPAHC...,1
2,ADDRCERMCQRYHDRREKKQCMKGCRYG,1
3,ADPTFGFTPLGLSEKANLQIMKAYD,1
4,AEEMGKGEVTISLRCKTKTECLKNIACEACVDCRCDKGICKCHGFT...,1


In [5]:
df_test = pd.read_excel(test)

In [6]:
df_test.head()

,sequence,label
0,ADIKRCVVDVKLSKPCTFQECIPLCFQRYNGNGVCTGKKNEICTCAYNC,1
1,AEIKKFSYDHCFHLCVAGEYGSNECFVDCAQKGFWHGDCANRTEKD...,1
2,AKQHHGYKQKFH,1
3,AKRFFGYKRKFF,1
4,AKRHHGLNCAKGVLA,1


## Concatenar ambas secuencias

In [7]:
# Concatenar los dataframes
df_combined = pd.concat([df_train, df_test])

In [8]:
df_combined.head()

,sequence,label
0,ACFKFLGECGAVPFPGTNADCTSCCVGNFGSAVCAGRVEVEGGVKH...,1
1,ACFTFLGECGPEPFTGSNADCLACCVALYKSPPVCAGRVEGVPAHC...,1
2,ADDRCERMCQRYHDRREKKQCMKGCRYG,1
3,ADPTFGFTPLGLSEKANLQIMKAYD,1
4,AEEMGKGEVTISLRCKTKTECLKNIACEACVDCRCDKGICKCHGFT...,1


In [9]:
df_combined.shape

(1986, 2)

## Verificar duplicados

In [11]:
df_combined["sequence"].unique().shape

(1986,)

In [12]:
df = df_combined.drop_duplicates().copy()

In [13]:
df.shape

(1986, 2)

## Verificar canónicos

In [14]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [15]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["sequence"]]

In [16]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [17]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 1986


In [18]:
display(df_validas)

,sequence,label
0,ACFKFLGECGAVPFPGTNADCTSCCVGNFGSAVCAGRVEVEGGVKH...,1
1,ACFTFLGECGPEPFTGSNADCLACCVALYKSPPVCAGRVEGVPAHC...,1
2,ADDRCERMCQRYHDRREKKQCMKGCRYG,1
3,ADPTFGFTPLGLSEKANLQIMKAYD,1
4,AEEMGKGEVTISLRCKTKTECLKNIACEACVDCRCDKGICKCHGFT...,1
...,...,...
425,YFSVDPKLSKPGEPVFNRVVSLKSSFQLPKQMAKEAGKIAAEMEKI...,0
426,YLAELTTVYGPLSDKAFSEYMLNKDPEDIGPSDSASNDLPTKFAIK...,0
427,YLHQLEEAKKYDHRILGLKQELVSLHEWSPGSWFFLPHGALIYNKL...,0
428,YNEGWIERHKKLGLIVVPVNEDDD,0


In [19]:
df_validas.to_csv('UniDL4BioPep_labeled.csv', index=False, header=True)

## Obtener metadatos

In [20]:
import json

In [21]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [22]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [23]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [24]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "UniDL4BioPep")

In [25]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [26]:
export_json("metadata.json", dict_metadata)